In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-01-10 18:54
@Project  :   Hands-on Deep Learning with PyTorch-tensor_broadcast_computing
张量的广播和科学运算
'''

# 导入所需的库
import torch

作为PyTorch中执行深度学习的基本数据类型，张量(Tensor) 也拥有非常多的数学运算函数和方法，以及对应的一系列计算规则。在PyTorch中， 能够作用与Tensor的运算，被统称作为算子。并且相比于NumPy, PyTorch给出了更加规范的算子（运算）的分类，从而方便用户在不同场景下调用不同类型的算子（运算）。

PyTorch总共为Tensor设计了六大类数学运算，分别是：
1. **逐点运算**（Pointwise Ops）：指的是针对Tensor中每个元素执行的相同运算操作
2. **规约运算**（Reduction Ops）：指的是对于某一张量进行操作得出某种总结值
3. **比较运算**（Comparison Ops）：指的是对多个张量进行比较运算的相关方法
4. 谱运算（Spectral Ops）：指的是涉及信号处理傅里叶变化的操作
5. **BLAS和LAPACK运算**：指的是基础线性代数程序集（Basic Linear Algeria Subprograms）和线性代数包（Linear Algeria Package）中定义的、主要用于线性代数科学计算的函数和方法
6. 其他运算（Other Ops）：其他未被归类的数学运算
## 1.张量的广播
张量的一个重要运算规则就是广播机制，即张量具备和NumPy相同的广播特性，也就是允许不同形状的张量之间进行计算。

In [2]:
# 1.相同形状的张量计算：相同形状的张量计算，尽管是对应位置元素进行计算，但本质上也是应用到了广播特性
t1 = torch.arange(3)
t1

tensor([0, 1, 2])

In [3]:
t1 + t1  # 对应位置元素分别相加

tensor([0, 2, 4])

In [4]:
# 2.不同形状的张量计算：广播是指在不同形状的张量进行计算时，一个或多个张量通过隐式转化，转化成相同形状的两个张量，从而完成计算
# 2.1 标量和任意形状的张量：标量和张量的每一个元素进行计算
t1 + 3, t1 + torch.tensor(3)

(tensor([3, 4, 5]), tensor([3, 4, 5]))

In [5]:
# 二维+零维
t2 = torch.randn(3, 4)
t2, t2 * 3

(tensor([[ 0.7150, -0.7346, -2.0608,  0.4908],
         [ 0.0555, -1.8117,  0.3407,  0.5190],
         [ 0.1214, -1.4022,  0.9097, -0.9274]]),
 tensor([[ 2.1451, -2.2039, -6.1824,  1.4725],
         [ 0.1666, -5.4352,  1.0221,  1.5570],
         [ 0.3641, -4.2065,  2.7290, -2.7823]]))

In [6]:
# 2.2 相同维度、不同形状的张量之间计算：在不同的维度上，只要其中有一个张量的形状值为1，就可以进行计算
t3 = torch.ones(1, 4)
t2.shape, t3.shape

(torch.Size([3, 4]), torch.Size([1, 4]))

In [7]:
t2 + t3  # 广播相当于将t3的形状(1, 4)拓展成了t2的(3, 4)，即复制了第一行3次，然后二者进行相加。也可以理解成t3的第一行和t2的3行分别进行了相加

tensor([[ 1.7150,  0.2654, -1.0608,  1.4908],
        [ 1.0555, -0.8117,  1.3407,  1.5190],
        [ 1.1214, -0.4022,  1.9097,  0.0726]])

In [8]:
t4 = torch.randint(10, size=(3, 1))
t4.shape, t4

(torch.Size([3, 1]),
 tensor([[1],
         [9],
         [6]]))

In [9]:
# torch.rand(2, 4) + t2  # RuntimeError，此时两个张量的形状第一个分量维度不同，但二者取值均不为1，因此无法广播
t2 + t4  # 将t4的形状(3, 1)拓展成了t2的(3, 4)，即复制了第一列4次，然后二者进行相加。也可以理解成t4的第一列和t2的4列分别进行了相加

tensor([[ 1.7150,  0.2654, -1.0608,  1.4908],
        [ 9.0555,  7.1883,  9.3407,  9.5190],
        [ 6.1214,  4.5978,  6.9097,  5.0726]])

In [10]:
t5 = torch.full((1, 5), 3.14)
t4.shape, t5.shape

(torch.Size([3, 1]), torch.Size([1, 5]))

In [11]:
t4 - t5  # 此时，t4的形状是(3, 1)，而t5的形状是(1, 5)，二者的形状在两个分量上均不相同，但都有存在1的情况，因此也可以广播

tensor([[-2.1400, -2.1400, -2.1400, -2.1400, -2.1400],
        [ 5.8600,  5.8600,  5.8600,  5.8600,  5.8600],
        [ 2.8600,  2.8600,  2.8600,  2.8600,  2.8600]])

In [12]:
# 三维张量的广播
t6 = torch.rand(3, 4, 5)
t7 = torch.ones(3, 4, 1)
t6, t7

(tensor([[[0.2208, 0.7407, 0.5660, 0.6648, 0.2483],
          [0.4753, 0.3959, 0.8526, 0.1421, 0.6915],
          [0.6056, 0.5682, 0.7772, 0.6718, 0.9016],
          [0.6123, 0.5647, 0.8968, 0.3973, 0.4021]],
 
         [[0.2180, 0.6825, 0.9867, 0.6720, 0.1055],
          [0.0322, 0.7212, 0.3004, 0.6613, 0.5593],
          [0.4931, 0.2389, 0.2589, 0.7177, 0.4261],
          [0.5830, 0.7284, 0.4142, 0.4883, 0.2152]],
 
         [[0.0023, 0.8295, 0.5410, 0.4303, 0.4247],
          [0.7969, 0.2112, 0.6365, 0.0840, 0.2401],
          [0.3753, 0.5698, 0.7700, 0.7090, 0.6642],
          [0.4343, 0.8155, 0.4198, 0.3962, 0.5768]]]),
 tensor([[[1.],
          [1.],
          [1.],
          [1.]],
 
         [[1.],
          [1.],
          [1.],
          [1.]],
 
         [[1.],
          [1.],
          [1.],
          [1.]]]))

In [13]:
t6 + t7

tensor([[[1.2208, 1.7407, 1.5660, 1.6648, 1.2483],
         [1.4753, 1.3959, 1.8526, 1.1421, 1.6915],
         [1.6056, 1.5682, 1.7772, 1.6718, 1.9016],
         [1.6123, 1.5647, 1.8968, 1.3973, 1.4021]],

        [[1.2180, 1.6825, 1.9867, 1.6720, 1.1055],
         [1.0322, 1.7212, 1.3004, 1.6613, 1.5593],
         [1.4931, 1.2389, 1.2589, 1.7177, 1.4261],
         [1.5830, 1.7284, 1.4142, 1.4883, 1.2152]],

        [[1.0023, 1.8295, 1.5410, 1.4303, 1.4247],
         [1.7969, 1.2112, 1.6365, 1.0840, 1.2401],
         [1.3753, 1.5698, 1.7700, 1.7090, 1.6642],
         [1.4343, 1.8155, 1.4198, 1.3962, 1.5768]]])

In [14]:
t8 = torch.randn(3, 1, 5)
t6 + t8

tensor([[[-1.1333,  1.1203,  1.3974, -0.6828,  0.5656],
         [-0.8789,  0.7754,  1.6841, -1.2055,  1.0088],
         [-0.7486,  0.9478,  1.6087, -0.6758,  1.2189],
         [-0.7419,  0.9442,  1.7282, -0.9504,  0.7194]],

        [[ 0.1671,  1.0046,  0.6083,  0.7349,  0.2266],
         [-0.0187,  1.0432, -0.0780,  0.7241,  0.6803],
         [ 0.4422,  0.5609, -0.1195,  0.7805,  0.5471],
         [ 0.5321,  1.0504,  0.0358,  0.5512,  0.3363]],

        [[ 0.7510,  1.5803, -1.6187, -0.1288,  1.1721],
         [ 1.5455,  0.9619, -1.5232, -0.4752,  0.9874],
         [ 1.1239,  1.3205, -1.3897,  0.1498,  1.4115],
         [ 1.1830,  1.5663, -1.7399, -0.1630,  1.3241]]])

In [15]:
# 两个张量的形状上有多个分量不同时，只要不同的分量仍然有一个取值为1，仍然可以广播
t9 = torch.full((3, 1, 1), 2.72)
t6 + t9

tensor([[[2.9408, 3.4607, 3.2860, 3.3848, 2.9683],
         [3.1953, 3.1159, 3.5726, 2.8621, 3.4115],
         [3.3256, 3.2882, 3.4972, 3.3918, 3.6216],
         [3.3323, 3.2847, 3.6168, 3.1173, 3.1221]],

        [[2.9380, 3.4025, 3.7067, 3.3920, 2.8255],
         [2.7522, 3.4412, 3.0204, 3.3813, 3.2793],
         [3.2131, 2.9589, 2.9789, 3.4377, 3.1461],
         [3.3030, 3.4484, 3.1342, 3.2083, 2.9352]],

        [[2.7223, 3.5495, 3.2610, 3.1503, 3.1447],
         [3.5169, 2.9312, 3.3565, 2.8040, 2.9601],
         [3.0953, 3.2898, 3.4900, 3.4290, 3.3842],
         [3.1543, 3.5355, 3.1398, 3.1162, 3.2968]]])

In [16]:
t7.shape, t8.shape, t7 + t8

(torch.Size([3, 4, 1]),
 torch.Size([3, 1, 5]),
 tensor([[[-0.3542,  1.3796,  1.8315, -0.3476,  1.3173],
          [-0.3542,  1.3796,  1.8315, -0.3476,  1.3173],
          [-0.3542,  1.3796,  1.8315, -0.3476,  1.3173],
          [-0.3542,  1.3796,  1.8315, -0.3476,  1.3173]],
 
         [[ 0.9491,  1.3220,  0.6216,  1.0628,  1.1210],
          [ 0.9491,  1.3220,  0.6216,  1.0628,  1.1210],
          [ 0.9491,  1.3220,  0.6216,  1.0628,  1.1210],
          [ 0.9491,  1.3220,  0.6216,  1.0628,  1.1210]],
 
         [[ 1.7486,  1.7508, -1.1597,  0.4409,  1.7473],
          [ 1.7486,  1.7508, -1.1597,  0.4409,  1.7473],
          [ 1.7486,  1.7508, -1.1597,  0.4409,  1.7473],
          [ 1.7486,  1.7508, -1.1597,  0.4409,  1.7473]]]))

In [17]:
# 3.不同维度的张量计算过程中广播：对于不同维度的张量，首先可以将低维的张量升维，然后依据相同维度不同形状的张量广播规则进行广播。而低维向量的升维也非常简单，只需将更高维度方向的形状填充为1即可
t10 = torch.arange(4).reshape(2, 2)
# 升维
t10.reshape(1, 2, 2), t10.reshape(1, 2, 2, 1)

(tensor([[[0, 1],
          [2, 3]]]),
 tensor([[[[0],
           [1]],
 
          [[2],
           [3]]]]))

In [18]:
t11 = torch.arange(12).reshape(3, 2, 2)
t10.shape, t11.shape

(torch.Size([2, 2]), torch.Size([3, 2, 2]))

In [19]:
t10 + t11  # t11和t10的计算过程，就相当于形状为(1,  2,  2)和(3,  2,  2)的两个张量进行计算，等价于它t10.reshape(1, 2, 2) + t11

tensor([[[ 0,  2],
         [ 4,  6]],

        [[ 4,  6],
         [ 8, 10]],

        [[ 8, 10],
         [12, 14]]])

In [20]:
torch.ones(2, 1) + torch.randn(3, 2, 3)  # 先转化为(2, 3) + (3, 2, 3)，再转化为(1, 2, 3) + (3, 2, 3)，即(3, 2, 3) + (3, 2, 3)

tensor([[[-0.0524,  1.3935,  1.4626],
         [ 1.5077,  1.6674,  0.1293]],

        [[-0.2937,  1.3470,  0.6516],
         [ 2.7063,  0.1476,  0.7693]],

        [[ 1.2229,  1.4061,  2.4156],
         [ 0.5272,  1.2750,  2.6202]]])

## 2.逐点运算(Pointwise Ops)
PyTorch中逐点运算大部分都是可以针对Tensor中每个元素都进行的数学科学运算，并且都是较为通用的数学科学运算，和NumPy中针对Array的科学运算类似，主要包括**数学基本运算**、**数值调整运算**和**数据科学运算**三类。

In [21]:
# 1.数学基本运算
a = torch.arange(12).reshape(3, 4)
b = torch.randn(3, 4)
a, b

(tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[-0.3718,  0.1998,  0.1194,  0.2817],
         [-0.8314, -0.1660, -0.2867,  1.2886],
         [ 1.8632, -0.0052,  0.5404,  0.4298]]))

In [22]:
# 加
torch.add(a, b), a + b

(tensor([[-0.3718,  1.1998,  2.1194,  3.2817],
         [ 3.1686,  4.8340,  5.7133,  8.2886],
         [ 9.8632,  8.9948, 10.5404, 11.4298]]),
 tensor([[-0.3718,  1.1998,  2.1194,  3.2817],
         [ 3.1686,  4.8340,  5.7133,  8.2886],
         [ 9.8632,  8.9948, 10.5404, 11.4298]]))

In [23]:
# 减
torch.subtract(a, b), a - b

(tensor([[ 0.3718,  0.8002,  1.8806,  2.7183],
         [ 4.8314,  5.1660,  6.2867,  5.7114],
         [ 6.1368,  9.0052,  9.4596, 10.5702]]),
 tensor([[ 0.3718,  0.8002,  1.8806,  2.7183],
         [ 4.8314,  5.1660,  6.2867,  5.7114],
         [ 6.1368,  9.0052,  9.4596, 10.5702]]))

In [24]:
# 乘
torch.multiply(a, b), a * b

(tensor([[-0.0000,  0.1998,  0.2388,  0.8451],
         [-3.3256, -0.8300, -1.7205,  9.0201],
         [14.9060, -0.0470,  5.4042,  4.7278]]),
 tensor([[-0.0000,  0.1998,  0.2388,  0.8451],
         [-3.3256, -0.8300, -1.7205,  9.0201],
         [14.9060, -0.0470,  5.4042,  4.7278]]))

In [25]:
# 除
torch.divide(a, b), a / b

(tensor([[   -0.0000,     5.0042,    16.7480,    10.6494],
         [   -4.8111,   -30.1222,   -20.9245,     5.4323],
         [    4.2936, -1723.8047,    18.5043,    25.5933]]),
 tensor([[   -0.0000,     5.0042,    16.7480,    10.6494],
         [   -4.8111,   -30.1222,   -20.9245,     5.4323],
         [    4.2936, -1723.8047,    18.5043,    25.5933]]))

In [26]:
# 2.数值调整运算
t12 = torch.randn(3, 4) * 5
t12

tensor([[ 4.8705, -8.4479, -0.5310, -3.9156],
        [ 8.5954, -1.5184,  6.8548,  4.4535],
        [-1.6154, -2.7859,  3.1104, -1.0522]])

In [27]:
# 虽然这些函数是数值调整函数，但并不会对原对象进行调整，而是输出新的结果
torch.abs(t12), torch.ceil(t12), torch.floor(t12), torch.round(t12), torch.neg(t12), t12  # 绝对值、向上取整、向下取整、四舍五入取整、相反数

(tensor([[4.8705, 8.4479, 0.5310, 3.9156],
         [8.5954, 1.5184, 6.8548, 4.4535],
         [1.6154, 2.7859, 3.1104, 1.0522]]),
 tensor([[ 5., -8., -0., -3.],
         [ 9., -1.,  7.,  5.],
         [-1., -2.,  4., -1.]]),
 tensor([[ 4., -9., -1., -4.],
         [ 8., -2.,  6.,  4.],
         [-2., -3.,  3., -2.]]),
 tensor([[ 5., -8., -1., -4.],
         [ 9., -2.,  7.,  4.],
         [-2., -3.,  3., -1.]]),
 tensor([[-4.8705,  8.4479,  0.5310,  3.9156],
         [-8.5954,  1.5184, -6.8548, -4.4535],
         [ 1.6154,  2.7859, -3.1104,  1.0522]]),
 tensor([[ 4.8705, -8.4479, -0.5310, -3.9156],
         [ 8.5954, -1.5184,  6.8548,  4.4535],
         [-1.6154, -2.7859,  3.1104, -1.0522]]))

In [28]:
# 若要对原对象本身进行修改，则可考虑使用方法_()的表达形式，对对象本身进行修改，表示原地操作
t13, t14 = t12.clone(), t12.clone()
t13, t14

(tensor([[ 4.8705, -8.4479, -0.5310, -3.9156],
         [ 8.5954, -1.5184,  6.8548,  4.4535],
         [-1.6154, -2.7859,  3.1104, -1.0522]]),
 tensor([[ 4.8705, -8.4479, -0.5310, -3.9156],
         [ 8.5954, -1.5184,  6.8548,  4.4535],
         [-1.6154, -2.7859,  3.1104, -1.0522]]))

In [29]:
t13.abs_(), t13, t14.floor_(), t14

(tensor([[4.8705, 8.4479, 0.5310, 3.9156],
         [8.5954, 1.5184, 6.8548, 4.4535],
         [1.6154, 2.7859, 3.1104, 1.0522]]),
 tensor([[4.8705, 8.4479, 0.5310, 3.9156],
         [8.5954, 1.5184, 6.8548, 4.4535],
         [1.6154, 2.7859, 3.1104, 1.0522]]),
 tensor([[ 4., -9., -1., -4.],
         [ 8., -2.,  6.,  4.],
         [-2., -3.,  3., -2.]]),
 tensor([[ 4., -9., -1., -4.],
         [ 8., -2.,  6.,  4.],
         [-2., -3.,  3., -2.]]))

In [30]:
# PyTorch的很多函数都有对应的原地操作函数
t13.unsqueeze_(0).shape, t13.shape

(torch.Size([1, 3, 4]), torch.Size([1, 3, 4]))

In [31]:
# 3.数据科学运篡
# 幂运算，expm1和log1p用于处理非常接近0的数
# torch.exp(3)  # TypeError，科学计算的对象只能是Tensor
#
torch.exp(t13), torch.expm1(t13), torch.exp2(t13), torch.pow(t13, 2), torch.sqrt(t13), torch.square(t13)

(tensor([[[1.3039e+02, 4.6652e+03, 1.7006e+00, 5.0179e+01],
          [5.4068e+03, 4.5649e+00, 9.4846e+02, 8.5930e+01],
          [5.0297e+00, 1.6215e+01, 2.2430e+01, 2.8640e+00]]]),
 tensor([[[1.2939e+02, 4.6642e+03, 7.0058e-01, 4.9179e+01],
          [5.4058e+03, 3.5649e+00, 9.4746e+02, 8.4930e+01],
          [4.0297e+00, 1.5215e+01, 2.1430e+01, 1.8640e+00]]]),
 tensor([[[ 29.2535, 349.1935,   1.4449,  15.0907],
          [386.7923,   2.8647, 115.7475,  21.9103],
          [  3.0639,   6.8968,   8.6362,   2.0737]]]),
 tensor([[[23.7221, 71.3667,  0.2819, 15.3319],
          [73.8812,  2.3055, 46.9888, 19.8340],
          [ 2.6094,  7.7614,  9.6746,  1.1072]]]),
 tensor([[[2.2069, 2.9065, 0.7287, 1.9788],
          [2.9318, 1.2322, 2.6182, 2.1103],
          [1.2710, 1.6691, 1.7636, 1.0258]]]),
 tensor([[[23.7221, 71.3667,  0.2819, 15.3319],
          [73.8812,  2.3055, 46.9888, 19.8340],
          [ 2.6094,  7.7614,  9.6746,  1.1072]]]))

In [32]:
# 老版本的PyTorch中，张量的大多数科学运算具有一定的静态性，对输入的张量类型有明确的要求，例如部分函数只能输入浮点型张量，而不能输入整型张量，会报错RuntimeError
t10, t10.dtype, torch.exp(t10), t11.dtype, t11.sqrt()

(tensor([[0, 1],
         [2, 3]]),
 torch.int64,
 tensor([[ 1.0000,  2.7183],
         [ 7.3891, 20.0855]]),
 torch.int64,
 tensor([[[0.0000, 1.0000],
          [1.4142, 1.7321]],
 
         [[2.0000, 2.2361],
          [2.4495, 2.6458]],
 
         [[2.8284, 3.0000],
          [3.1623, 3.3166]]]))

In [33]:
# 对数运算
torch.log(t13), torch.log10(t13), torch.log2(t13), torch.log1p(t13)

(tensor([[[ 1.5832,  2.1339, -0.6331,  1.3650],
          [ 2.1512,  0.4177,  1.9250,  1.4937],
          [ 0.4796,  1.0246,  1.1348,  0.0509]]]),
 tensor([[[ 0.6876,  0.9267, -0.2749,  0.5928],
          [ 0.9343,  0.1814,  0.8360,  0.6487],
          [ 0.2083,  0.4450,  0.4928,  0.0221]]]),
 tensor([[[ 2.2841,  3.0786, -0.9133,  1.9692],
          [ 3.1036,  0.6025,  2.7771,  2.1550],
          [ 0.6919,  1.4782,  1.6371,  0.0735]]]),
 tensor([[[1.7699, 2.2458, 0.4259, 1.5924],
          [2.2613, 0.9236, 2.0611, 1.6963],
          [0.9614, 1.3313, 1.4135, 0.7189]]]))

In [34]:
# 幂运算和对数运算的关系
t10, torch.exp(torch.log(t10)), t11, torch.exp2(torch.log2(t11))

(tensor([[0, 1],
         [2, 3]]),
 tensor([[0., 1.],
         [2., 3.]]),
 tensor([[[ 0,  1],
          [ 2,  3]],
 
         [[ 4,  5],
          [ 6,  7]],
 
         [[ 8,  9],
          [10, 11]]]),
 tensor([[[ 0.0000,  1.0000],
          [ 2.0000,  3.0000]],
 
         [[ 4.0000,  5.0000],
          [ 6.0000,  7.0000]],
 
         [[ 8.0000,  9.0000],
          [10.0000, 11.0000]]]))

In [35]:
# 三角函数运算
t15 = torch.arange(0, torch.pi * 12, torch.pi / 2).reshape(4, 6)
t15, torch.sin(t15), torch.cos(t15), torch.tan(t15)

(tensor([[ 0.0000,  1.5708,  3.1416,  4.7124,  6.2832,  7.8540],
         [ 9.4248, 10.9956, 12.5664, 14.1372, 15.7080, 17.2788],
         [18.8496, 20.4204, 21.9911, 23.5619, 25.1327, 26.7035],
         [28.2743, 29.8451, 31.4159, 32.9867, 34.5575, 36.1283]]),
 tensor([[ 0.0000e+00,  1.0000e+00, -8.7423e-08, -1.0000e+00,  1.7485e-07,
           1.0000e+00],
         [-2.3850e-08, -1.0000e+00,  3.4969e-07,  1.0000e+00, -6.7553e-07,
          -1.0000e+00],
         [ 4.7700e-08,  1.0000e+00,  5.8013e-07, -1.0000e+00,  6.9938e-07,
           1.0000e+00],
         [-7.1549e-08, -1.0000e+00, -5.5628e-07,  1.0000e+00,  1.1841e-06,
          -1.0000e+00]]),
 tensor([[ 1.0000e+00, -4.3711e-08, -1.0000e+00,  1.1925e-08,  1.0000e+00,
           1.3907e-07],
         [-1.0000e+00, -2.9007e-07,  1.0000e+00, -3.5775e-08, -1.0000e+00,
          -5.9206e-07],
         [ 1.0000e+00, -6.8746e-07, -1.0000e+00,  5.9624e-08,  1.0000e+00,
           5.6821e-07],
         [-1.0000e+00,  7.1131e-07,  1.0000

In [36]:
# 排序运算
# 升序
t8, torch.sort(t8), t8.sort()

(tensor([[[-1.3542,  0.3796,  0.8315, -1.3476,  0.3173]],
 
         [[-0.0509,  0.3220, -0.3784,  0.0628,  0.1210]],
 
         [[ 0.7486,  0.7508, -2.1597, -0.5591,  0.7473]]]),
 torch.return_types.sort(
 values=tensor([[[-1.3542, -1.3476,  0.3173,  0.3796,  0.8315]],
 
         [[-0.3784, -0.0509,  0.0628,  0.1210,  0.3220]],
 
         [[-2.1597, -0.5591,  0.7473,  0.7486,  0.7508]]]),
 indices=tensor([[[0, 3, 4, 1, 2]],
 
         [[2, 0, 3, 4, 1]],
 
         [[2, 3, 4, 0, 1]]])),
 torch.return_types.sort(
 values=tensor([[[-1.3542, -1.3476,  0.3173,  0.3796,  0.8315]],
 
         [[-0.3784, -0.0509,  0.0628,  0.1210,  0.3220]],
 
         [[-2.1597, -0.5591,  0.7473,  0.7486,  0.7508]]]),
 indices=tensor([[[0, 3, 4, 1, 2]],
 
         [[2, 0, 3, 4, 1]],
 
         [[2, 3, 4, 0, 1]]])))

In [37]:
# 降序排列
values, indices = torch.sort(t8, descending=True)
values, indices

(tensor([[[ 0.8315,  0.3796,  0.3173, -1.3476, -1.3542]],
 
         [[ 0.3220,  0.1210,  0.0628, -0.0509, -0.3784]],
 
         [[ 0.7508,  0.7486,  0.7473, -0.5591, -2.1597]]]),
 tensor([[[2, 1, 4, 3, 0]],
 
         [[1, 4, 3, 0, 2]],
 
         [[1, 0, 4, 3, 2]]]))

## 3.规约计算
规约运算是针对某张量进行某种总结，最后得出一个具体总结值的函数，此类函数主要包含了数据科学领域内的诸多统计分析函数，如均值、极值、方差、中位数函数等。

In [38]:
# 均值，方差，标准差，方差和均值，标准差和均值
torch.mean(t15), torch.var(t15), torch.std(t15), torch.var_mean(t15), torch.std_mean(t15)

(tensor(18.0642),
 tensor(123.3700),
 tensor(11.1072),
 (tensor(123.3700), tensor(18.0642)),
 (tensor(11.1072), tensor(18.0642)))

In [39]:
# 最大值，最大值索引，最小值，最小值索引，中位数
torch.max(t15), torch.argmax(t15), torch.min(t15), torch.argmin(t15), torch.median(t15)

(tensor(36.1283), tensor(23), tensor(0.), tensor(0), tensor(17.2788))

In [40]:
# 求和，累乘，各元素求和（适用于小数据量），获取张量中最大的k个值
torch.sum(t14), torch.prod(t14), torch.logsumexp(t14, -1), torch.topk(t14, 2, 0)

(tensor(2.),
 tensor(-1990656.),
 tensor([4.0071, 8.1430, 3.0158]),
 torch.return_types.topk(
 values=tensor([[ 8., -2.,  6.,  4.],
         [ 4., -3.,  3., -2.]]),
 indices=tensor([[1, 1, 1, 1],
         [0, 2, 2, 2]])))

In [41]:
# dist函数计算闵氏距离，p=1为曼哈顿距离，p=2时为欧式距离
# 欧式距离
torch.dist(t12, t14, p=2), torch.sqrt(torch.square(t12 - t14).sum())

(tensor(1.9791), tensor(1.9791))

In [42]:
# 曼哈顿距离
torch.dist(t12, t14, p=1), torch.sum((t12 - t14).abs())

(tensor(6.0184), tensor(6.0184))

In [43]:
# 指定维度进行计算
torch.sum(t12, 0), torch.sum(t12, 1)  # 分别按列求和、按行求和

(tensor([ 11.8506, -12.7522,   9.4343,  -0.5143]),
 tensor([-8.0239, 18.3854, -2.3431]))

In [44]:
torch.mean(t6), torch.mean(t6, dim=0), t6.mean(1)

(tensor(0.5138),
 tensor([[0.1471, 0.7509, 0.6979, 0.5891, 0.2595],
         [0.4348, 0.4428, 0.5965, 0.2958, 0.4969],
         [0.4913, 0.4590, 0.6021, 0.6995, 0.6640],
         [0.5432, 0.7029, 0.5769, 0.4272, 0.3981]]),
 tensor([[0.4785, 0.5674, 0.7731, 0.4690, 0.5609],
         [0.3316, 0.5928, 0.4900, 0.6348, 0.3265],
         [0.4022, 0.6065, 0.5918, 0.4049, 0.4764]]))

In [45]:
torch.sort(t6), torch.sort(t6, dim=0), t6.sort(1), t6.sort(dim=2, descending=True)

(torch.return_types.sort(
 values=tensor([[[0.2208, 0.2483, 0.5660, 0.6648, 0.7407],
          [0.1421, 0.3959, 0.4753, 0.6915, 0.8526],
          [0.5682, 0.6056, 0.6718, 0.7772, 0.9016],
          [0.3973, 0.4021, 0.5647, 0.6123, 0.8968]],
 
         [[0.1055, 0.2180, 0.6720, 0.6825, 0.9867],
          [0.0322, 0.3004, 0.5593, 0.6613, 0.7212],
          [0.2389, 0.2589, 0.4261, 0.4931, 0.7177],
          [0.2152, 0.4142, 0.4883, 0.5830, 0.7284]],
 
         [[0.0023, 0.4247, 0.4303, 0.5410, 0.8295],
          [0.0840, 0.2112, 0.2401, 0.6365, 0.7969],
          [0.3753, 0.5698, 0.6642, 0.7090, 0.7700],
          [0.3962, 0.4198, 0.4343, 0.5768, 0.8155]]]),
 indices=tensor([[[0, 4, 2, 3, 1],
          [3, 1, 0, 4, 2],
          [1, 0, 3, 2, 4],
          [3, 4, 1, 0, 2]],
 
         [[4, 0, 3, 1, 2],
          [0, 2, 4, 3, 1],
          [1, 2, 4, 0, 3],
          [4, 2, 3, 0, 1]],
 
         [[0, 4, 3, 2, 1],
          [3, 1, 4, 2, 0],
          [0, 1, 4, 3, 2],
          [3, 2, 0, 4, 

## 4.比较运算
比较运算是一类较为简单的运算类型，和Python原生的布尔运算类似，常用于不同张量之间的逻辑运算，最终返回逻辑运算结果（逻辑类型张量)。

In [46]:
# 比较两个张量的各元素是否相等
c = torch.randn(3, 4)
d = torch.rand(3, 4)
torch.eq(c, d), c == d

(tensor([[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]))

In [47]:
# 判断2个张量是否是相同的张量：对象类型相同、各个元素相同
torch.equal(c, d)

False

In [48]:
# 比较第一个张量的元素是否大于第二个张量
torch.gt(c, d), c > d

(tensor([[ True, False, False, False],
         [False, False, False,  True],
         [False, False, False, False]]),
 tensor([[ True, False, False, False],
         [False, False, False,  True],
         [False, False, False, False]]))

In [49]:
# 比较第一个张量的元素是否小于第二个张量
torch.lt(c, d), c < d

(tensor([[False,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]),
 tensor([[False,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]))

In [50]:
# 比较第一个张量的元素是否大于等于第二个张量
torch.ge(c, d), c >= d

(tensor([[ True, False, False, False],
         [False, False, False,  True],
         [False, False, False, False]]),
 tensor([[ True, False, False, False],
         [False, False, False,  True],
         [False, False, False, False]]))

In [51]:
# 比较第一个张量的元素是否小于等于第二个张量
torch.le(c, d), c <= d

(tensor([[False,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]),
 tensor([[False,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]))

In [52]:
# 比较两个张量的元素是否不相等
torch.ne(c, d), c != d

(tensor([[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]]),
 tensor([[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]]))